In [1]:
import pdbufr
import sys
import traceback
 
from math import isnan

from eccodes import *
from ecmwf.opendata import Client
from math import isnan

from ipywidgets import interact
import os
import birdy
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
import ipyleaflet
import ipywidgets as widgets

from Magics import macro as magics
from IPython.display import display
from Magics.macro import *

import numpy as np
import xarray as xr

import haversine as hs

import warnings
warnings.filterwarnings("ignore")

from utils import *

In [2]:
## CODE TRIALS WITH FORECAST OF THE 17th AUGUST ##

start_date_forecast = datetime(2023, 8, 17, 0)

In [3]:
## DOWNLOAD DATA FORECAST ##
# If too early in the morning the forecast of the current day cannot be donwloaded, need to fix this -> If raiseError: download forecast of the previous day
client = Client(source="azure")

start_date = start_date_forecast.strftime("%Y%m%d")

client.retrieve(
    date=int(start_date),
    time=0,
    stream="enfo",
    type="tf",
    step=240,
    target=f"data/tc_test_track_data.bufr",
);

20230817000000-240h-enfo-tf.bufr:   0%|          | 0.00/752k [00:00<?, ?B/s]

In [4]:
## LOAD TRACKS FORECAST DATA IN A DATAFRAME ##
df_storms_forecast = create_storms_df()

## LOAD OBSERVED TRACKS IN A DATAFRAME
df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])

print('Observed tracks: ', df_storms_observed.NAME.squeeze().unique().tolist())
print('Forecasted tracks: ', df_storms_forecast.stormIdentifier.unique())

Observed tracks:  ['KHANUN', 'DORA', 'LAN', 'FERNANDA', 'GREG', 'HILARY']
Forecasted tracks:  ['07E' '08E' '09E' '09W']


In [5]:
## FIND THE CORRESPONDENT OBSERVED STORMS TO THOSE PRESENT IN THE FORECAST ##
storms_pair = storms_pair(df_storms_forecast, df_storms_observed)
print(storms_pair)

['07E-FERNANDA', '08E-GREG', '09E-HILARY', '09W-LAN']


In [6]:
## WIDGETS FOR INTERACTIVE MAPS PLOT ##

# Cyclone dropdown selection
cyclone = widgets.Dropdown(
    options = storms_pair,
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'

# Longitude slider
longitude = np.arange(-180,185,5)
longitude_slider = widgets.SelectionRangeSlider(
    options=longitude,
    index=(0, len(longitude)-1),
    description='Longitude:',
    orientation='horizontal',
    layout={'width': '400px'},
    disabled=False,
)

longitude_slider.style.description_width = '68px'
longitude_slider.style.handle_color = 'lightgreen'

# Latitude slider
latitude = np.arange(-90,95,5)
latitude_slider = widgets.SelectionRangeSlider(
    options=latitude,
    index=(0, len(latitude)-1),
    description='Latitude:',
    orientation='horizontal',
    layout={'width': '400px'},
    disabled=False,
)

latitude_slider.style.description_width = '58px'
latitude_slider.style.handle_color = 'lightgreen'

In [7]:
# widgets.interactive(plot_cyclone_tracks_magics, cyclone=cyclone, lat_boundaries=latitude_slider, lon_boundaries=longitude_slider)

In [8]:
widgets.interactive(plot_cyclone_tracks_ipyleaflet, cyclone=cyclone)

interactive(children=(Dropdown(description='Active Storms:', options=('07E-FERNANDA', '08E-GREG', '09E-HILARY'…